In [1]:
import pandas as pd
import numpy as np
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import os

Using TensorFlow backend.


In [2]:
word2TagDict = {}
dictTags = {}
data = []

with open('final.tsv', 'r') as train_data:
    tagLine = train_data.readline()
    while tagLine:
        tokens = tagLine.split()
        if len(tokens) == 6:
            if tokens[0] in dictTags:
                dictTags[tokens[0]][tokens[4]] = tokens[5]
            else:
                dictTags[tokens[0]] = {}
                dictTags[tokens[0]][tokens[4]] = tokens[5]

            word2TagDict[tokens[4]] = tokens[5]
        tagLine = train_data.readline()


In [3]:
data = []

for tweet in dictTags:
    sentence = dictTags[tweet]
    listTup = [(k, v) for k, v in sentence.items()]
    data.append(listTup)

In [4]:
len(dictTags)

8732

In [5]:
words = list(set(word2TagDict.keys()))
tags = list(set(word2TagDict.values()))
words.sort()
tags.sort()

In [6]:
num_tags = len(tags)
num_words = len(words)

In [7]:
tags

['ambiguous', 'lang1', 'lang2', 'mixed', 'ne', 'other']

In [8]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [9]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in data]

In [10]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=num_words-1)

In [22]:
y = [[tag2idx[w[1]] for w in s] for s in data]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["other"])

In [23]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=num_tags) for i in y]

In [24]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [25]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [25]:
# input = Input(shape=(max_len,))
# model = Embedding(input_dim=num_words + 1, output_dim=50,
#                   input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
# model = Bidirectional(LSTM(units=50, return_sequences=True,
#                            recurrent_dropout=0.1))(model)  # variational biLSTM
# model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
# crf = CRF(num_tags)  # CRF layer
# out = crf(model)  # output

In [26]:
## biLSTM for language identification 
input = Input(shape=(max_len,))
model = Embedding(input_dim=num_words, output_dim=50, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)  # softmax output layer


In [27]:
model = Model(input, out)

In [27]:
# model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/home/jharsh/miniconda3/envs/kerasGPU/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/jharsh/miniconda3/envs/kerasGPU/lib/python3.6/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [28]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [29]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=50, validation_split=0.1, verbose=1)


Train on 7072 samples, validate on 786 samples
Epoch 1/50
7072/7072 [==============================] - 24s 3ms/step - loss: 0.1632 - acc: 0.9434 - val_loss: 0.0611 - val_acc: 0.9816
Epoch 2/50
7072/7072 [==============================] - 23s 3ms/step - loss: 0.0480 - acc: 0.9866 - val_loss: 0.0505 - val_acc: 0.9858
Epoch 3/50
7072/7072 [==============================] - 23s 3ms/step - loss: 0.0393 - acc: 0.9894 - val_loss: 0.0457 - val_acc: 0.9872
Epoch 4/50
7072/7072 [==============================] - 23s 3ms/step - loss: 0.0342 - acc: 0.9910 - val_loss: 0.0443 - val_acc: 0.9882
Epoch 5/50
7072/7072 [==============================] - 23s 3ms/step - loss: 0.0302 - acc: 0.9925 - val_loss: 0.0425 - val_acc: 0.9887
Epoch 6/50
7072/7072 [==============================] - 23s 3ms/step - loss: 0.0268 - acc: 0.9936 - val_loss: 0.0421 - val_acc: 0.9891
Epoch 7/50
7072/7072 [==============================] - 23s 3ms/step - loss: 0.0240 - acc: 0.9944 - val_loss: 0.0422 - val_acc: 0.9893
Epoch 8/

In [30]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
test_pred = model.predict(X_te, verbose=1)

idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "other"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

874/874 [==============================] - 1s 1ms/step


In [31]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

      lang1       0.62      0.66      0.64      1115
  ambiguous       0.04      0.09      0.05        22
      other       0.84      0.86      0.85      1843
      lang2       0.56      0.54      0.55       648
         ne       0.33      0.42      0.37       220
      mixed       0.00      0.00      0.00         2

avg / total       0.70      0.72      0.71      3850

